In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data.csv')

In [3]:
def parse_budget_data_numeric(x):
    return float(x.replace(",", ""))

In [4]:
numeric_cols = ['ProductionBudget', 'DomesticGross', 'WorldwideGross']
for numeric_col in numeric_cols:
    df[numeric_col] = df[numeric_col].apply(parse_budget_data_numeric)

In [5]:
def calculate_profit(x):
    x['Profit'] = x['WorldwideGross'] - x['ProductionBudget']
    x['IsProfitable'] = x['Profit'] > 0
    return x

df = df.apply(calculate_profit, axis=1)

In [ ]:
def contains_word_extractor(x):
    contains_word_features = {}
    try:
        with open('movie_scripts/' + x['fuzzy_picked_file']) as f:
            text = f.read()
            words = text.split()
            for word in words:
                contains_word_features['contains_word_' + word + '.html'] = True
        x['contains_word_features'] = contains_word_features
    except FileNotFoundError:
        print("German, I cannot find this file: ", x['fuzzy_picked_file'])
    return x

df = df.apply(contains_word_extractor, axis=1)

In [7]:
from fuzzywuzzy import fuzz

from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir("movie_scripts") if isfile(join("movie_scripts", f))]

def retrieve_fuzzy_file(x):
    best_fuzzy = 0
    best_fuzzy_file = None
    for file in onlyfiles:
        ratio = fuzz.ratio(x['MovieTitle'], file)
        if ratio > best_fuzzy:
            best_fuzzy = ratio
            best_fuzzy_file = file

    x['fuzzy_picked_file'] = best_fuzzy_file
    return x

df = df.apply(retrieve_fuzzy_file, axis=1)

/Users/varuntandon/Github/CS221FinalProject/221venv/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
